In [1]:
import meshio
import numpy as np
from multiprocessing import Pool
import time

from ugs import *

In [2]:
infile = "australia/data/TAS.vtk"
outfile = "costed_tas.vtk"
mesh = meshio.read(infile)

max_distance = 250000

In [3]:
# We need to define a way to calculate cost
def travel_cost(mesh, current, _next):
    # The travel_cost can be any function, including just the distance.
    # Here, we exagerate the elevation difference, to make changing elevation more costly
    if current == _next:
        return 0
    
    z_scaling = 100.  # 100. is a random number to pick, but has quite a big impact on the resulting paths.
    
    new_current = np.append(mesh.points[current][:2], mesh.point_data['Z'][current] * z_scaling)
    new_next    = np.append(mesh.points[_next][:2],   mesh.point_data['Z'][_next]   * z_scaling)
    
    return int(np.linalg.norm(new_current - new_next))  # return as Int, just for niceness

## Prepare for parallel execution

We want to use a parallel map function to calculate the cost for all points. We can make this easier by 'baking in' the parameters of a function we know, and leave only the `starting point` as a variable.

In [4]:
from functools import partial

# We bake in the mesh and travel_cost function into the get_from_point function
get_from_point_in_mesh = partial(get_from_point, mesh = mesh, travel_cost_function = travel_cost, max_distance = max_distance)

In [5]:
# Now we can use the get_from_point function with only a point ID as a parameter, by going via the new get_from_point_in_mesh function
# Here we do a test, and see the output format: (point, total cost of all paths to that point)
print get_from_point_in_mesh(2087)

(2087, 0)


## Prepare input data

We don't want to calculate the LEC of a point below sea-level, so here we find all the points above sea-level, so they can be used as starting points.

In [6]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0][::-1]

In [7]:
print "Total starting points available: ", points_above_sealevel.shape[0]

Total starting points available:  1093


In [8]:
# Output data variables
all_costs = []
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

# For parallel execution, we chunk the data up into increments (inc), so we can monitor the progress
start = 0
inc = 20
stop = inc

# Run on 11 CPUs
p = Pool(11)

while start <  points_above_sealevel.shape[0]-1:
    start_time = time.time()
    # Here we use a parallel map function to send out the chunk across the CPUs in the Pool
    costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:stop])
    print "From ", start, " to ", stop, " took ", time.time() - start_time, "seconds, starting with point", points_above_sealevel[start], ". Percent complete: ", 100*(float(stop)/points_above_sealevel.shape[0])
    
    # Save the data
    all_costs.extend(costs)
    
    # Write out data progressively, so we can see progress in Paraview
    for i in all_costs:
        mesh.point_data['cost'][i[0]] = i[1]
    meshio.write(outfile, mesh)
    
    # move to the next chunk of data
    start += inc
    stop += inc
    if stop >= points_above_sealevel.shape[0]:
        stop = points_above_sealevel.shape[0]-1
    

From  0  to  20  took  20.3182940483 seconds, starting with point 1233 . Percent complete:  1.82982616651
From  20  to  40  took  21.2341079712 seconds, starting with point 1213 . Percent complete:  3.65965233303
From  40  to  60  took  19.9485881329 seconds, starting with point 1193 . Percent complete:  5.48947849954
From  60  to  80  took  21.0873069763 seconds, starting with point 1173 . Percent complete:  7.31930466606
From  80  to  100  took  19.5123109818 seconds, starting with point 1153 . Percent complete:  9.14913083257
From  100  to  120  took  21.1051311493 seconds, starting with point 1133 . Percent complete:  10.9789569991
From  120  to  140  took  23.2081868649 seconds, starting with point 1113 . Percent complete:  12.8087831656
From  140  to  160  took  21.9765648842 seconds, starting with point 1093 . Percent complete:  14.6386093321
From  160  to  180  took  20.0348408222 seconds, starting with point 1073 . Percent complete:  16.4684354986
From  180  to  200  took  26.

## Output

Show some of the results:

In [9]:
print all_costs[:100]

[(1233, 61337505), (1232, 61724591), (1231, 57889317), (1230, 60903776), (1229, 57004828), (1228, 64623103), (1227, 63541893), (1226, 60051747), (1225, 63295727), (1224, 58647942), (1223, 66066340), (1222, 62483698), (1221, 57919376), (1220, 57585105), (1219, 65441707), (1218, 65128496), (1217, 59938628), (1216, 63510391), (1215, 57496759), (1214, 57969656), (1213, 65367376), (1212, 57369074), (1211, 61330194), (1210, 60232263), (1209, 69297839), (1208, 60617585), (1207, 61036617), (1206, 63090062), (1205, 60304364), (1204, 59389483), (1203, 61829417), (1202, 67019246), (1201, 55511454), (1200, 62120226), (1199, 66399861), (1198, 57799342), (1197, 61309103), (1196, 63300118), (1195, 55010626), (1194, 59679487), (1193, 56891619), (1192, 59230449), (1191, 67880306), (1190, 56480323), (1189, 60947831), (1188, 66433157), (1187, 56597869), (1186, 64412322), (1185, 63370058), (1184, 68961587), (1183, 55386412), (1182, 67346831), (1181, 62361414), (1180, 64901588), (1179, 62655951), (1178, 58